# Prueba 

In [54]:
import pandas as pd
import numpy as np


# Cargar el archivo CSV
file_path = '../../../data/processed/df_train.csv'
df = pd.read_csv(file_path)

def transform_dataset(df):

    # Convertir las fechas a formato datetime
    df['ApprovalDate'] = pd.to_datetime(df['ApprovalDate'], errors='coerce')
    df['DisbursementDate'] = pd.to_datetime(df['DisbursementDate'], errors='coerce')

    # Generar solo las nuevas columnas
    df_train = pd.DataFrame()

    # 1. 'approval_month' - Mes de aprobación
    df_train['approval_month'] = df['ApprovalDate'].dt.month

    # 2. 'Bank_grouped_bin' - Codificación por frecuencia
    bank_frequency = df['Bank'].value_counts(normalize=True).to_dict()  # Frecuencia de cada banco
    df_train['Bank_grouped_freq'] = df['Bank'].map(bank_frequency)  # Mapeamos la frecuencia

    # 3. 'City_grouped_bin' - Codificación por frecuencia
    city_frequency = df['City'].value_counts(normalize=True).to_dict()  # Frecuencia de cada ciudad
    df_train['City_grouped_freq'] = df['City'].map(city_frequency) 

    # 3. 'job_ratio' - Relación entre el número de empleados y el número de trabajos creados
    df_train['job_ratio'] = df['NoEmp'] / df['CreateJob'].replace(0, 1)  # Para evitar división por cero

    # 4. 'RetainedJob_bin' - Codificación binaria de si se retuvo empleo (0 o 1)
    df_train['RetainedJob_bin'] = (df['RetainedJob'] > 0).astype(int)

    # 5. 'funding_ratio' - Relación entre la cantidad de financiamiento y la cantidad total de préstamos
    df_train['funding_ratio'] = df['BalanceGross'].replace({'\$': '', ',': ''}, regex=True).astype(float) / df['DisbursementGross'].replace({'\$': '', ',': ''}, regex=True).astype(float)

    # 6. 'LowDoc' - Binarización de LowDoc (si LowDoc es 1)
    df_train['LowDoc'] = (df['LowDoc'] == 1).astype(int)

    # 7. 'NewExist' - Binarización de NewExist (si NewExist es 1 o 0)
    df_train['NewExist'] = df['NewExist'].fillna(0).astype(int)

    # 8. 'approval_year' - Año de aprobación
    df_train['approval_year'] = df['ApprovalDate'].dt.year

    # 9. 'is_franchise' - Binarización de si es franquicia
    df_train['is_franchise'] = (df['FranchiseCode'] == 1).astype(int)

    # 10. 'CreateJob_bin' - Binarización de si hubo creación de trabajo (si CreateJob > 0)
    df_train['CreateJob_bin'] = (df['CreateJob'] > 0).astype(int)

    # 11. 'NoEmp_bin_code' - Binarización de si hay empleados
    df_train['NoEmp_bin_code'] = (df['NoEmp'] > 0).astype(int)

    # 12. 'disbursement_month' - Mes de desembolso
    df_train['disbursement_month'] = df['DisbursementDate'].dt.month

    # 13. 'approval_dayofweek' - Día de la semana de la aprobación (0 = Lunes, 6 = Domingo)
    df_train['approval_dayofweek'] = df['ApprovalDate'].dt.dayofweek

    # 14. 'UrbanRural' - Codificación de tipo urbano/rural
    df_train['UrbanRural'] = df['UrbanRural'].astype(int)

    # 15. 'approval_season' - Estacionalidad (1: Invierno, 2: Primavera, 3: Verano, 4: Otoño)
    df_train['approval_season'] = df_train['approval_month'].apply(lambda x: (x % 12 + 3) // 3)



    # 17. 'RevLineCr' - Binarización de si el crédito es Revolving Line of Credit
    df_train['RevLineCr'] = (df['RevLineCr'] == 1).astype(int)

    # 18. 'disbursement_dayofweek' - Día de la semana del desembolso
    df_train['disbursement_dayofweek'] = df['DisbursementDate'].dt.dayofweek

    # 19. 'days_to_disbursement' - Diferencia en días entre la aprobación y el desembolso
    df_train['days_to_disbursement'] = (df['DisbursementDate'] - df['ApprovalDate']).dt.days

    # 20. 'BalanceGross_bin_code' - Binarización de BalanceGross (si es mayor que 0)
    df_train['BalanceGross_bin_code'] = (df['BalanceGross'] > 0).astype(int)

    # 21. 'retention_ratio' - Relación entre trabajos retenidos y trabajos creados
    df_train['retention_ratio'] = df['RetainedJob'] / df['CreateJob'].replace(0, 1)

    # 22. 'DisbursementGross_bin_code' - Binarización de DisbursementGross (si es mayor que 0)
    df_train['DisbursementGross_bin_code'] = (df['DisbursementGross'] > 0).astype(int)


    # Eliminar cualquier valor NaN en el DataFrame
    fillna_value=-9999
    df_train = df_train.fillna(fillna_value)

    # Reemplazar cualquier valor infinito por 0 (si existiera alguno)
    df_train.replace([np.inf, -np.inf], 0, inplace=True)

    df_train['Accept'] = df['Accept']

    # Devolver el nuevo DataFrame con solo las nuevas columnas
    return df_train

# Usar la función para crear un nuevo df_train con las nuevas columnas
df_train = transform_dataset(df)

# Mostrar las primeras filas del dataframe transformado
print(df_train.head())


   approval_month  Bank_grouped_freq  City_grouped_freq  job_ratio  \
0              11           0.075936           0.001839        2.0   
1               6           0.034027           0.013400        2.0   
2               3           0.127042           0.000219        0.5   
3               6           0.186687           0.076856        2.0   
4               4           0.186687           0.001708        0.0   

   RetainedJob_bin  funding_ratio  LowDoc  NewExist  approval_year  \
0                1            0.0       0         1           2005   
1                1            0.0       0         1           2005   
2                1            0.0       0         1           2003   
3                0            0.0       0         1           1995   
4                0            0.0       0         0           2009   

   is_franchise  ...  approval_dayofweek  UrbanRural  approval_season  \
0             0  ...                   1           1                4   
1           

In [41]:
# Cargar el archivo CSV
file_path = '../../../data/processed/df_test_nolabel.csv'
df = pd.read_csv(file_path)
# Usar la función para crear un nuevo df_train con las nuevas columnas
df_test = transform_dataset(df)
df_test['id'] = df['id']

# Mostrar las primeras filas del dataframe transformado
print(df_test.head())

   approval_month  Bank_grouped_bin  job_ratio  RetainedJob_bin  \
0              10                 1   9.000000                1   
1               4                 1   3.750000                1   
2               1                 1  13.000000                1   
3               7                 1   1.473684                1   
4              11                 1   1.176471                1   

   funding_ratio  LowDoc  NewExist  approval_year  is_franchise  \
0            0.0       0         0           2004             1   
1            0.0       0         0           2000             1   
2            0.0       0         0           2003             1   
3            0.0       0         1           2004             1   
4            0.0       0         0           1990             1   

   CreateJob_bin  ...  UrbanRural  approval_season  City_grouped_bin  \
0              0  ...           2                4                 1   
1              1  ...           1                2

In [42]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3284 entries, 0 to 3283
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   approval_month              3284 non-null   int32  
 1   Bank_grouped_bin            3284 non-null   int32  
 2   job_ratio                   3284 non-null   float64
 3   RetainedJob_bin             3284 non-null   int32  
 4   funding_ratio               3284 non-null   float64
 5   LowDoc                      3284 non-null   int32  
 6   NewExist                    3284 non-null   int32  
 7   approval_year               3284 non-null   int32  
 8   is_franchise                3284 non-null   int32  
 9   CreateJob_bin               3284 non-null   int32  
 10  NoEmp_bin_code              3284 non-null   int32  
 11  disbursement_month          3284 non-null   float64
 12  approval_dayofweek          3284 non-null   int32  
 13  UrbanRural                  3284 

## Modelo catboost

In [56]:
# Importar las bibliotecas necesarias
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix, f1_score
import pandas as pd

# Aquí asumimos que ya tienes tu DataFrame 'df_train' y la variable objetivo 'y_train'

# Separar características y variable objetivo
X = df_train[['approval_year', 'approval_month', 'approval_dayofweek', 'days_to_disbursement', 'approval_season', 'disbursement_month', 'disbursement_dayofweek']]
y = df_train['Accept']  # Reemplaza 'target_column' por el nombre de tu variable objetivo

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Debug 1: División en entrenamiento y prueba')

# Convertir variables categóricas a dummy variables
X_train_1 = pd.get_dummies(X_train, drop_first=True)
X_test_1 = pd.get_dummies(X_test, drop_first=True)
print('Debug 2: Xtrain_1, Xtest_1 get_dummies')

# Asegurarse de que ambos conjuntos de datos tengan las mismas columnas
train_columns = X_train_1.columns
X_test_1 = X_test_1.reindex(columns=train_columns, fill_value=0)
print('Debug 3: Xtrain_1, Xtest_1 ajustados')

# Aplicar SMOTE para equilibrar las clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_1, y_train = smote.fit_resample(X_train_1, y_train)
print('Debug 4: SMOTE aplicado a X_train_1, y_train')

# Crear y entrenar el modelo
model_1 = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.01,
    depth=5,
    l2_leaf_reg=3,
    loss_function='Logloss',
    od_type='Iter',  # Especifica que quieres usar early stopping iterativo
    od_wait=50,  # Utiliza este parámetro
    bagging_temperature=0.3,
    subsample=0.8,
    max_bin=255,
    min_data_in_leaf=10
)

model_1.fit(X_train_1, y_train)
print('Debug 5: model_1.fit hecho')

# Realizar predicciones
y_pred_1 = model_1.predict(X_test_1)
print('Debug 6: model_1.predict hecho')

# Evaluar el modelo
cm = confusion_matrix(y_test, y_pred_1)
f1 = f1_score(y_test, y_pred_1, average='macro')

# Imprimir las métricas
print("Matriz de Confusión:")
print(cm)
print("\nMacro F1 Score:", f1)


Debug 1: División en entrenamiento y prueba
Debug 2: Xtrain_1, Xtest_1 get_dummies
Debug 3: Xtrain_1, Xtest_1 ajustados
Debug 4: SMOTE aplicado a X_train_1, y_train
0:	learn: 0.6913285	total: 9.92ms	remaining: 9.91s
1:	learn: 0.6895846	total: 24ms	remaining: 12s
2:	learn: 0.6878968	total: 35.9ms	remaining: 11.9s
3:	learn: 0.6862955	total: 46.2ms	remaining: 11.5s
4:	learn: 0.6847030	total: 55.7ms	remaining: 11.1s
5:	learn: 0.6832270	total: 65.1ms	remaining: 10.8s
6:	learn: 0.6816158	total: 74.5ms	remaining: 10.6s
7:	learn: 0.6801733	total: 85.7ms	remaining: 10.6s
8:	learn: 0.6785252	total: 94.7ms	remaining: 10.4s
9:	learn: 0.6769037	total: 103ms	remaining: 10.2s
10:	learn: 0.6753246	total: 112ms	remaining: 10.1s
11:	learn: 0.6739212	total: 122ms	remaining: 10s
12:	learn: 0.6724277	total: 131ms	remaining: 9.96s
13:	learn: 0.6708286	total: 140ms	remaining: 9.88s
14:	learn: 0.6694129	total: 150ms	remaining: 9.83s
15:	learn: 0.6680502	total: 159ms	remaining: 9.8s
16:	learn: 0.6667339	total:

In [57]:


# Separar características y variable objetivo
X = df_train[['NewExist','NoEmp_bin_code',  'job_ratio', 'retention_ratio' ]]
y = df_train['Accept']  # Reemplaza 'target_column' por el nombre de tu variable objetivo

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Debug 1: División en entrenamiento y prueba')

# Convertir variables categóricas a dummy variables
X_train_2 = pd.get_dummies(X_train, drop_first=True)
X_test_2 = pd.get_dummies(X_test, drop_first=True)
print('Debug 2: Xtrain_1, Xtest_1 get_dummies')

# Asegurarse de que ambos conjuntos de datos tengan las mismas columnas
train_columns = X_train_2.columns
X_test_2 = X_test_2.reindex(columns=train_columns, fill_value=0)
print('Debug 3: Xtrain_1, Xtest_1 ajustados')

# Aplicar SMOTE para equilibrar las clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_2, y_train = smote.fit_resample(X_train_2, y_train)
print('Debug 4: SMOTE aplicado a X_train_1, y_train')

model_2 = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=5,
    l2_leaf_reg=7,
    loss_function='Logloss',
    od_type='Iter',  # Especifica que quieres usar early stopping iterativo
    od_wait=50,  # Utiliza este parámetro
    bagging_temperature=0.3,
    subsample=0.8,
    max_bin=255,
    min_data_in_leaf=10
)

model_2.fit(X_train_2, y_train)
print('Debug 5: model_1.fit hecho')

# Realizar predicciones
y_pred_2 = model_2.predict(X_test_2)
print('Debug 6: model_1.predict hecho')

# Evaluar el modelo
cm = confusion_matrix(y_test, y_pred_2)
f1 = f1_score(y_test, y_pred_2, average='macro')

# Imprimir las métricas
print("Matriz de Confusión:")
print(cm)
print("\nMacro F1 Score:", f1)


Debug 1: División en entrenamiento y prueba
Debug 2: Xtrain_1, Xtest_1 get_dummies
Debug 3: Xtrain_1, Xtest_1 ajustados
Debug 4: SMOTE aplicado a X_train_1, y_train
0:	learn: 0.6905228	total: 16ms	remaining: 16s
1:	learn: 0.6879676	total: 33.1ms	remaining: 16.5s
2:	learn: 0.6855484	total: 42.3ms	remaining: 14s
3:	learn: 0.6833944	total: 50.3ms	remaining: 12.5s
4:	learn: 0.6814756	total: 57.4ms	remaining: 11.4s
5:	learn: 0.6795330	total: 64.2ms	remaining: 10.6s
6:	learn: 0.6777590	total: 70.6ms	remaining: 10s
7:	learn: 0.6763404	total: 76.8ms	remaining: 9.52s
8:	learn: 0.6750171	total: 83ms	remaining: 9.13s
9:	learn: 0.6737038	total: 89.7ms	remaining: 8.88s
10:	learn: 0.6725366	total: 96ms	remaining: 8.63s
11:	learn: 0.6717356	total: 101ms	remaining: 8.33s
12:	learn: 0.6707456	total: 107ms	remaining: 8.14s
13:	learn: 0.6699508	total: 114ms	remaining: 8s
14:	learn: 0.6691450	total: 120ms	remaining: 7.86s
15:	learn: 0.6684023	total: 127ms	remaining: 7.82s
16:	learn: 0.6677053	total: 135ms

In [58]:



# Separar características y variable objetivo
X = df_train[['NewExist', 'is_franchise', 'UrbanRural', 'LowDoc', 'Bank_grouped_freq' , 'RevLineCr', 'City_grouped_freq' ]]

y = df_train['Accept']  # Reemplaza 'target_column' por el nombre de tu variable objetivo

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Debug 1: División en entrenamiento y prueba')

# Convertir variables categóricas a dummy variables
X_train_3 = pd.get_dummies(X_train, drop_first=True)
X_test_3= pd.get_dummies(X_test, drop_first=True)
print('Debug 2: Xtrain_1, Xtest_1 get_dummies')

# Asegurarse de que ambos conjuntos de datos tengan las mismas columnas
train_columns = X_train_3.columns
X_test_3 = X_test_3.reindex(columns=train_columns, fill_value=0)
print('Debug 3: Xtrain_1, Xtest_1 ajustados')

# Aplicar SMOTE para equilibrar las clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_3, y_train = smote.fit_resample(X_train_3, y_train)
print('Debug 4: SMOTE aplicado a X_train_1, y_train')

model_3 = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.01,
    depth=5,
    l2_leaf_reg=7,
    loss_function='Logloss',
    od_type='Iter',  # Especifica que quieres usar early stopping iterativo
    od_wait=50,  # Utiliza este parámetro
    bagging_temperature=0.3,
    subsample=0.8,
    max_bin=255,
    min_data_in_leaf=10
)

model_3.fit(X_train_3, y_train)
print('Debug 5: model_1.fit hecho')

# Realizar predicciones
y_pred_3 = model_3.predict(X_test_3)
print('Debug 6: model_1.predict hecho')

# Evaluar el modelo
cm = confusion_matrix(y_test, y_pred_3)
f1 = f1_score(y_test, y_pred_3, average='macro')

# Imprimir las métricas
print("Matriz de Confusión:")
print(cm)
print("\nMacro F1 Score:", f1)


Debug 1: División en entrenamiento y prueba
Debug 2: Xtrain_1, Xtest_1 get_dummies
Debug 3: Xtrain_1, Xtest_1 ajustados
Debug 4: SMOTE aplicado a X_train_1, y_train
0:	learn: 0.6919331	total: 7.61ms	remaining: 7.61s
1:	learn: 0.6910252	total: 13ms	remaining: 6.51s
2:	learn: 0.6898453	total: 20.8ms	remaining: 6.91s
3:	learn: 0.6887141	total: 28.2ms	remaining: 7.01s
4:	learn: 0.6875722	total: 36.7ms	remaining: 7.29s
5:	learn: 0.6864260	total: 43.7ms	remaining: 7.24s
6:	learn: 0.6853679	total: 50.8ms	remaining: 7.2s
7:	learn: 0.6842696	total: 58.3ms	remaining: 7.23s
8:	learn: 0.6832853	total: 64.7ms	remaining: 7.13s
9:	learn: 0.6822794	total: 72ms	remaining: 7.13s
10:	learn: 0.6812749	total: 80.4ms	remaining: 7.22s
11:	learn: 0.6802705	total: 87.6ms	remaining: 7.21s
12:	learn: 0.6792790	total: 94.7ms	remaining: 7.19s
13:	learn: 0.6783809	total: 101ms	remaining: 7.13s
14:	learn: 0.6773335	total: 109ms	remaining: 7.17s
15:	learn: 0.6764239	total: 119ms	remaining: 7.32s
16:	learn: 0.6755885	

In [59]:

# Separar características y variable objetivo
X = df_train[['DisbursementGross_bin_code', 'BalanceGross_bin_code', 'funding_ratio', 'disbursement_month','disbursement_dayofweek', 'days_to_disbursement', 'approval_year', 'approval_month', 'approval_dayofweek', 'approval_season' ]]

y = df_train['Accept']  # Reemplaza 'target_column' por el nombre de tu variable objetivo

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Debug 1: División en entrenamiento y prueba')

# Convertir variables categóricas a dummy variables
X_train_4 = pd.get_dummies(X_train, drop_first=True)
X_test_4 = pd.get_dummies(X_test, drop_first=True)
print('Debug 2: Xtrain_1, Xtest_1 get_dummies')

# Asegurarse de que ambos conjuntos de datos tengan las mismas columnas
train_columns = X_train_4.columns
X_test_4 = X_test_4.reindex(columns=train_columns, fill_value=0)
print('Debug 3: Xtrain_1, Xtest_1 ajustados')

# Aplicar SMOTE para equilibrar las clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_4, y_train = smote.fit_resample(X_train_4, y_train)
print('Debug 4: SMOTE aplicado a X_train_1, y_train')

model_4 = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.01,
    depth=5,
    l2_leaf_reg=7,
    loss_function='Logloss',
    od_type='Iter',  # Especifica que quieres usar early stopping iterativo
    od_wait=50,  # Utiliza este parámetro
    bagging_temperature=0.3,
    subsample=0.8,
    max_bin=255,
    min_data_in_leaf=10
)

model_4.fit(X_train_4, y_train)
print('Debug 5: model_1.fit hecho')

# Realizar predicciones
y_pred_4 = model_4.predict(X_test_4)
print('Debug 6: model_1.predict hecho')

# Evaluar el modelo
cm = confusion_matrix(y_test, y_pred_4)
f1 = f1_score(y_test, y_pred_4, average='macro')

# Imprimir las métricas
print("Matriz de Confusión:")
print(cm)
print("\nMacro F1 Score:", f1)

Debug 1: División en entrenamiento y prueba
Debug 2: Xtrain_1, Xtest_1 get_dummies
Debug 3: Xtrain_1, Xtest_1 ajustados
Debug 4: SMOTE aplicado a X_train_1, y_train
0:	learn: 0.6915485	total: 10.9ms	remaining: 10.9s
1:	learn: 0.6900464	total: 18.6ms	remaining: 9.29s
2:	learn: 0.6882895	total: 25ms	remaining: 8.31s
3:	learn: 0.6866799	total: 31.8ms	remaining: 7.91s
4:	learn: 0.6849621	total: 38.8ms	remaining: 7.72s
5:	learn: 0.6833969	total: 45.7ms	remaining: 7.57s
6:	learn: 0.6817976	total: 52.9ms	remaining: 7.5s
7:	learn: 0.6802198	total: 59.4ms	remaining: 7.36s
8:	learn: 0.6786752	total: 65.5ms	remaining: 7.21s
9:	learn: 0.6771220	total: 72.1ms	remaining: 7.13s
10:	learn: 0.6755982	total: 79.4ms	remaining: 7.14s
11:	learn: 0.6739678	total: 85.7ms	remaining: 7.05s
12:	learn: 0.6724831	total: 92ms	remaining: 6.99s
13:	learn: 0.6709869	total: 100ms	remaining: 7.05s
14:	learn: 0.6694855	total: 106ms	remaining: 6.99s
15:	learn: 0.6682051	total: 113ms	remaining: 6.98s
16:	learn: 0.6666982	

In [60]:

# Separar características y variable objetivo
X = df_train.drop(columns=['Accept'])

y = df_train['Accept']  # Reemplaza 'target_column' por el nombre de tu variable objetivo

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Debug 1: División en entrenamiento y prueba')

# Convertir variables categóricas a dummy variables
X_train_5 = pd.get_dummies(X_train, drop_first=True)
X_test_5 = pd.get_dummies(X_test, drop_first=True)
print('Debug 2: Xtrain_1, Xtest_1 get_dummies')

# Asegurarse de que ambos conjuntos de datos tengan las mismas columnas
train_columns = X_train_5.columns
X_test_5 = X_test_5.reindex(columns=train_columns, fill_value=0)
print('Debug 3: Xtrain_1, Xtest_1 ajustados')

# Aplicar SMOTE para equilibrar las clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_5, y_train = smote.fit_resample(X_train_5, y_train)
print('Debug 4: SMOTE aplicado a X_train_1, y_train')

model_5 = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.01,
    depth=5,
    l2_leaf_reg=7,
    loss_function='Logloss',
    od_type='Iter',  # Especifica que quieres usar early stopping iterativo
    od_wait=50,  # Utiliza este parámetro
    bagging_temperature=0.3,
    subsample=0.8,
    max_bin=255,
    min_data_in_leaf=10
)

model_5.fit(X_train_5, y_train)
print('Debug 5: model_1.fit hecho')

# Realizar predicciones
y_pred_5 = model_5.predict(X_test_5)
print('Debug 6: model_1.predict hecho')

# Evaluar el modelo
cm = confusion_matrix(y_test, y_pred_5)
f1 = f1_score(y_test, y_pred_5, average='macro')

# Imprimir las métricas
print("Matriz de Confusión:")
print(cm)
print("\nMacro F1 Score:", f1)

Debug 1: División en entrenamiento y prueba
Debug 2: Xtrain_1, Xtest_1 get_dummies
Debug 3: Xtrain_1, Xtest_1 ajustados
Debug 4: SMOTE aplicado a X_train_1, y_train
0:	learn: 0.6884131	total: 9.14ms	remaining: 9.13s
1:	learn: 0.6849213	total: 17.3ms	remaining: 8.63s
2:	learn: 0.6810443	total: 24.1ms	remaining: 8.01s
3:	learn: 0.6769948	total: 30.6ms	remaining: 7.61s
4:	learn: 0.6730760	total: 38.1ms	remaining: 7.59s
5:	learn: 0.6690312	total: 44.8ms	remaining: 7.43s
6:	learn: 0.6660586	total: 51ms	remaining: 7.24s
7:	learn: 0.6632592	total: 57.8ms	remaining: 7.17s
8:	learn: 0.6602414	total: 65ms	remaining: 7.15s
9:	learn: 0.6563745	total: 72.6ms	remaining: 7.19s
10:	learn: 0.6529648	total: 78.8ms	remaining: 7.09s
11:	learn: 0.6495858	total: 85.1ms	remaining: 7s
12:	learn: 0.6464969	total: 91.3ms	remaining: 6.93s
13:	learn: 0.6436250	total: 98.6ms	remaining: 6.95s
14:	learn: 0.6412665	total: 105ms	remaining: 6.91s
15:	learn: 0.6385408	total: 111ms	remaining: 6.85s
16:	learn: 0.6364192	t

In [61]:
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix, f1_score
import pandas as pd

# Separar características y variable objetivo
X = df_train.drop(columns=['Accept'])

y = df_train['Accept']  # Reemplaza 'target_column' por el nombre de tu variable objetivo

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Debug 1: División en entrenamiento y prueba')

# Convertir variables categóricas a dummy variables
X_train_6 = pd.get_dummies(X_train, drop_first=True)
X_test_6 = pd.get_dummies(X_test, drop_first=True)
print('Debug 2: Xtrain_1, Xtest_1 get_dummies')

# Asegurarse de que ambos conjuntos de datos tengan las mismas columnas
train_columns = X_train_6.columns
X_test_6 = X_test_6.reindex(columns=train_columns, fill_value=0)
print('Debug 3: Xtrain_1, Xtest_1 ajustados')

# Aplicar SMOTE para equilibrar las clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_6, y_train = smote.fit_resample(X_train_6, y_train)
print('Debug 4: SMOTE aplicado a X_train_1, y_train')

# Definir el ensemble (votación por mayoría)
ensemble_model = VotingClassifier(estimators=[
    ('model_1', model_1),
    ('model_2', model_2),
    ('model_4', model_4)
], voting='hard')  # 'hard' votación por mayoría

# Entrenar el ensemble
ensemble_model.fit(X_train_6, y_train)

# Hacer predicciones
y_pred_ensemble = ensemble_model.predict(X_test_6)

# Evaluar el modelo
cm_ensemble = confusion_matrix(y_test, y_pred_ensemble)
f1_ensemble = f1_score(y_test, y_pred_ensemble, average='macro')

# Imprimir las métricas
print("Matriz de Confusión del Ensemble:")
print(cm_ensemble)
print("\nMacro F1 Score del Ensemble:", f1_ensemble)

Debug 1: División en entrenamiento y prueba
Debug 2: Xtrain_1, Xtest_1 get_dummies
Debug 3: Xtrain_1, Xtest_1 ajustados
Debug 4: SMOTE aplicado a X_train_1, y_train
0:	learn: 0.6880860	total: 6.89ms	remaining: 6.89s
1:	learn: 0.6844564	total: 14.5ms	remaining: 7.25s
2:	learn: 0.6803508	total: 21.6ms	remaining: 7.19s
3:	learn: 0.6760853	total: 28ms	remaining: 6.97s
4:	learn: 0.6719689	total: 35.1ms	remaining: 6.99s
5:	learn: 0.6675470	total: 41.9ms	remaining: 6.94s
6:	learn: 0.6645427	total: 48.2ms	remaining: 6.83s
7:	learn: 0.6616951	total: 54.6ms	remaining: 6.77s
8:	learn: 0.6585513	total: 61.1ms	remaining: 6.72s
9:	learn: 0.6544942	total: 67.9ms	remaining: 6.72s
10:	learn: 0.6511803	total: 74.4ms	remaining: 6.69s
11:	learn: 0.6480686	total: 81.2ms	remaining: 6.68s
12:	learn: 0.6447792	total: 87.4ms	remaining: 6.64s
13:	learn: 0.6422807	total: 93.5ms	remaining: 6.59s
14:	learn: 0.6398990	total: 99.8ms	remaining: 6.55s
15:	learn: 0.6370623	total: 107ms	remaining: 6.59s
16:	learn: 0.634

In [62]:
import datetime
import pandas as pd

def create_submission(
    model,  # Acepta el modelo entrenado (ensemble_model en tu caso)
    test_df: pd.DataFrame,  # DataFrame de test
    submissions_folder: str,  # Carpeta donde guardar los archivos de submission
    submission_name: str  # Nombre del archivo de salida
):
    # Extraer los ids para el archivo final
    test_ids = test_df["id"]

    # Eliminar la columna 'id' para que el modelo solo use las características
    test_prediction_columns = test_df.columns.to_list()
    test_prediction_columns.remove("id")
    
    # Seleccionar las columnas para predicción
    test_prediction_data = test_df[test_prediction_columns]

    # Realizar la predicción
    predictions = model.predict(test_prediction_data)

    # Crear el DataFrame de las predicciones con 'id' y 'Accept'
    prediction_df = pd.DataFrame({
        "id": test_ids,
        "Accept": predictions
    })

    # Guardar las predicciones en un archivo CSV
    prediction_df.to_csv(f"{submissions_folder}/{submission_name}", sep=",", index=False)

    print(f"✅ Submission generada correctamente: '{submissions_folder}/{submission_name}'")

# Llamada a la función para generar la submission
create_submission(
    model=ensemble_model,
    test_df=df_test,
    submissions_folder='.',  # Reemplazar con la ruta correcta
    submission_name=f"voting_classifier_submission_{datetime.datetime.now().strftime('%Y%m%d')}.csv"
)


CatBoostError: catboost/libs/data/model_dataset_compatibility.cpp:81: At position 1 should be feature with name Bank_grouped_freq (found Bank_grouped_bin).

## Modelo 2

In [ ]:

# 4. Búsqueda de Hiperparámetros con GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 400, 800],
    'max_depth': [10, 30, 50, 80, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2', None],
    'class_weight': [None, 'balanced']
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='f1_macro')
grid_search.fit(X_train_res, y_train_res)
print("Mejores parámetros:", grid_search.best_params_)

# 5. Modelo con mejores parámetros
best_rf = grid_search.best_estimator_

# 6. Entrenamiento con los mejores parámetros
best_rf.fit(X_train_res, y_train_res)


In [ ]:
# Importar las bibliotecas necesarias
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from sklearn.metrics import confusion_matrix, f1_score
import pandas as pd

# Aquí asumimos que ya tienes tu DataFrame 'df_train' y la variable objetivo 'y_train'

# Separar características y variable objetivo
X = df_train[['approval_year', 'approval_month', 'approval_dayofweek', 'days_to_disbursement', 'approval_season', 'disbursement_month', 'disbursement_dayofweek']]
y = df_train['Accept']  # Reemplaza 'Accept' por el nombre de tu variable objetivo

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Debug 1: División en entrenamiento y prueba')

# Convertir variables categóricas a dummy variables
X_train_1 = pd.get_dummies(X_train, drop_first=True)
X_test_1 = pd.get_dummies(X_test, drop_first=True)
print('Debug 2: Xtrain_1, Xtest_1 get_dummies')

# Asegurarse de que ambos conjuntos de datos tengan las mismas columnas
train_columns = X_train_1.columns
X_test_1 = X_test_1.reindex(columns=train_columns, fill_value=0)
print('Debug 3: Xtrain_1, Xtest_1 ajustados')

# Balanceo de clases con SMOTE en entrenamiento
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_1, y_train)
print('Debug 4: SMOTE aplicado a X_train, y_train')

# Búsqueda de Hiperparámetros con GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 400, 800],
    'max_depth': [10, 30, 50, 80, None],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1],
    'colsample_bytree': [0.8, 0.9, 1]
}

xg_model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
grid_search = GridSearchCV(estimator=xg_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='f1_macro')
grid_search.fit(X_train_res, y_train_res)
print("Mejores parámetros:", grid_search.best_params_)

# Modelo con mejores parámetros
best_xgb = grid_search.best_estimator_

# Entrenamiento con los mejores parámetros
best_xgb.fit(X_train_res, y_train_res)

# Predicciones
y_pred = best_xgb.predict(X_test_1)

print('Debug 5: model_1.predict hecho')

# Evaluar el modelo
cm = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

# Imprimir las métricas
print("Matriz de Confusión:")
print(cm)
print("\nMacro F1 Score:", f1)


Debug 1: División en entrenamiento y prueba
Debug 2: Xtrain_1, Xtest_1 get_dummies
Debug 3: Xtrain_1, Xtest_1 ajustados
Debug 4: SMOTE aplicado a X_train, y_train
Fitting 5 folds for each of 720 candidates, totalling 3600 fits


In [ ]:

# Separar características y variable objetivo
X = df_train[['NewExist','NoEmp_bin_code',  'job_ratio', 'retention_ratio' ]]
y = df_train['Accept']  # Reemplaza 'target_column' por el nombre de tu variable objetivo

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Debug 1: División en entrenamiento y prueba')

# Convertir variables categóricas a dummy variables
X_train_2 = pd.get_dummies(X_train, drop_first=True)
X_test_2 = pd.get_dummies(X_test, drop_first=True)
print('Debug 2: Xtrain_1, Xtest_1 get_dummies')

# Asegurarse de que ambos conjuntos de datos tengan las mismas columnas
train_columns = X_train_2.columns
X_test_2 = X_test_2.reindex(columns=train_columns, fill_value=0)
print('Debug 3: Xtrain_1, Xtest_1 ajustados')

# Balanceo de clases con SMOTE en entrenamiento
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_2, y_train)
print('Debug 4: SMOTE aplicado a X_train, y_train')

# Búsqueda de Hiperparámetros con GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 400, 800],
    'max_depth': [10, 30, 50, 80, None],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1],
    'colsample_bytree': [0.8, 0.9, 1]
}

xg_model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
grid_search = GridSearchCV(estimator=xg_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='f1_macro')
grid_search.fit(X_train_res, y_train_res)
print("Mejores parámetros:", grid_search.best_params_)

# Modelo con mejores parámetros
best_xgb = grid_search.best_estimator_

# Entrenamiento con los mejores parámetros
best_xgb.fit(X_train_res, y_train_res)

# Predicciones
y_pred = best_xgb.predict(X_test_1)

print('Debug 5: model_1.predict hecho')

# Evaluar el modelo
cm = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

# Imprimir las métricas
print("Matriz de Confusión:")
print(cm)
print("\nMacro F1 Score:", f1)


In [ ]:

# Aquí asumimos que ya tienes tu DataFrame 'df_train' y la variable objetivo 'y_train'

# Separar características y variable objetivo
# Separar características y variable objetivo
X = df_train[['NewExist', 'is_franchise', 'UrbanRural', 'LowDoc', 'Bank_grouped_freq' , 'RevLineCr', 'City_grouped_freq' ]]

y = df_train['Accept']  
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Debug 1: División en entrenamiento y prueba')

# Convertir variables categóricas a dummy variables
X_train_3 = pd.get_dummies(X_train, drop_first=True)
X_test_3 = pd.get_dummies(X_test, drop_first=True)
print('Debug 2: Xtrain_1, Xtest_1 get_dummies')

# Asegurarse de que ambos conjuntos de datos tengan las mismas columnas
train_columns = X_train_3.columns
X_test_3 = X_test_3.reindex(columns=train_columns, fill_value=0)
print('Debug 3: Xtrain_1, Xtest_1 ajustados')

# Balanceo de clases con SMOTE en entrenamiento
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_3, y_train)
print('Debug 4: SMOTE aplicado a X_train, y_train')

# Búsqueda de Hiperparámetros con GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 400, 800],
    'max_depth': [10, 30, 50, 80, None],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1],
    'colsample_bytree': [0.8, 0.9, 1]
}

xg_model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
grid_search = GridSearchCV(estimator=xg_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='f1_macro')
grid_search.fit(X_train_res, y_train_res)
print("Mejores parámetros:", grid_search.best_params_)

# Modelo con mejores parámetros
best_xgb = grid_search.best_estimator_

# Entrenamiento con los mejores parámetros
best_xgb.fit(X_train_res, y_train_res)

# Predicciones
y_pred = best_xgb.predict(X_test_1)

print('Debug 5: model_1.predict hecho')

# Evaluar el modelo
cm = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

# Imprimir las métricas
print("Matriz de Confusión:")
print(cm)
print("\nMacro F1 Score:", f1)


In [ ]:

# Aquí asumimos que ya tienes tu DataFrame 'df_train' y la variable objetivo 'y_train'

# Separar características y variable objetivo
# Separar características y variable objetivo
X = df_train[['DisbursementGross_bin_code', 'BalanceGross_bin_code', 'funding_ratio', 'disbursement_month','disbursement_dayofweek', 'days_to_disbursement', 'approval_year', 'approval_month', 'approval_dayofweek', 'approval_season' ]]

y = df_train['Accept']  
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Debug 1: División en entrenamiento y prueba')

# Convertir variables categóricas a dummy variables
X_train_4 = pd.get_dummies(X_train, drop_first=True)
X_test_4 = pd.get_dummies(X_test, drop_first=True)
print('Debug 2: Xtrain_1, Xtest_1 get_dummies')

# Asegurarse de que ambos conjuntos de datos tengan las mismas columnas
train_columns = X_train_4.columns
X_test_4 = X_test_4.reindex(columns=train_columns, fill_value=0)
print('Debug 3: Xtrain_1, Xtest_1 ajustados')

# Balanceo de clases con SMOTE en entrenamiento
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_4, y_train)
print('Debug 4: SMOTE aplicado a X_train, y_train')

# Búsqueda de Hiperparámetros con GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 400, 800],
    'max_depth': [10, 30, 50, 80, None],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1],
    'colsample_bytree': [0.8, 0.9, 1]
}

xg_model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
grid_search = GridSearchCV(estimator=xg_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='f1_macro')
grid_search.fit(X_train_res, y_train_res)
print("Mejores parámetros:", grid_search.best_params_)

# Modelo con mejores parámetros
best_xgb = grid_search.best_estimator_

# Entrenamiento con los mejores parámetros
best_xgb.fit(X_train_res, y_train_res)

# Predicciones
y_pred = best_xgb.predict(X_test_1)

print('Debug 5: model_1.predict hecho')

# Evaluar el modelo
cm = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

# Imprimir las métricas
print("Matriz de Confusión:")
print(cm)
print("\nMacro F1 Score:", f1)


In [ ]:

# Separar características y variable objetivo
X = df_train.drop(columns=['Accept'])
y = df_train['Accept']  # Reemplaza 'target_column' por el nombre de tu variable objetivo

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Debug 1: División en entrenamiento y prueba')

# Convertir variables categóricas a dummy variables
X_train_5 = pd.get_dummies(X_train, drop_first=True)
X_test_5 = pd.get_dummies(X_test, drop_first=True)
print('Debug 2: Xtrain_1, Xtest_1 get_dummies')

# Asegurarse de que ambos conjuntos de datos tengan las mismas columnas
train_columns = X_train_5.columns
X_test_5 = X_test_5.reindex(columns=train_columns, fill_value=0)
print('Debug 3: Xtrain_1, Xtest_1 ajustados')

# Balanceo de clases con SMOTE en entrenamiento
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_5, y_train)
print('Debug 4: SMOTE aplicado a X_train, y_train')

# Búsqueda de Hiperparámetros con GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 400, 800],
    'max_depth': [10, 30, 50, 80, None],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1],
    'colsample_bytree': [0.8, 0.9, 1]
}


xg_model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
grid_search = GridSearchCV(estimator=xg_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='f1_macro')
grid_search.fit(X_train_res, y_train_res)
print("Mejores parámetros:", grid_search.best_params_)

# Modelo con mejores parámetros
best_xgb = grid_search.best_estimator_

# Entrenamiento con los mejores parámetros
best_xgb.fit(X_train_res, y_train_res)

# Predicciones
y_pred = best_xgb.predict(X_test_1)

print('Debug 5: model_1.predict hecho')

# Evaluar el modelo
cm = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

# Imprimir las métricas
print("Matriz de Confusión:")
print(cm)
print("\nMacro F1 Score:", f1)

In [ ]:

# Separar características y variable objetivo
X = df_train.drop(columns=['Accept'])

y = df_train['Accept']  # Reemplaza 'target_column' por el nombre de tu variable objetivo

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Debug 1: División en entrenamiento y prueba')

# Convertir variables categóricas a dummy variables
X_train_6 = pd.get_dummies(X_train, drop_first=True)
X_test_6 = pd.get_dummies(X_test, drop_first=True)
print('Debug 2: Xtrain_1, Xtest_1 get_dummies')

# Asegurarse de que ambos conjuntos de datos tengan las mismas columnas
train_columns = X_train_6.columns
X_test_6 = X_test_6.reindex(columns=train_columns, fill_value=0)
print('Debug 3: Xtrain_1, Xtest_1 ajustados')

# Aplicar SMOTE para equilibrar las clases en el conjunto de entrenamiento
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_6, y_train = smote.fit_resample(X_train_6, y_train)
print('Debug 4: SMOTE aplicado a X_train_1, y_train')

# Definir el ensemble (votación por mayoría)
ensemble_model = VotingClassifier(estimators=[
    ('model_1', model_1),
    ('model_2', model_2),
    ('model_3', model_3),
    ('model_4', model_4),
    ('model_5', model_5)
], voting='hard')  # 'hard' votación por mayoría

# Entrenar el ensemble
ensemble_model.fit(X_train_6, y_train)

# Hacer predicciones
y_pred_ensemble = ensemble_model.predict(X_test_6)

# Evaluar el modelo
cm_ensemble = confusion_matrix(y_test, y_pred_ensemble)
f1_ensemble = f1_score(y_test, y_pred_ensemble, average='macro')

# Imprimir las métricas
print("Matriz de Confusión del Ensemble:")
print(cm_ensemble)
print("\nMacro F1 Score del Ensemble:", f1_ensemble)


In [ ]:
import datetime
import pandas as pd

def create_submission(
    model,  # Acepta el modelo entrenado (ensemble_model en tu caso)
    test_df: pd.DataFrame,  # DataFrame de test
    submissions_folder: str,  # Carpeta donde guardar los archivos de submission
    submission_name: str  # Nombre del archivo de salida
):
    # Extraer los ids para el archivo final
    test_ids = test_df["id"]

    # Eliminar la columna 'id' para que el modelo solo use las características
    test_prediction_columns = test_df.columns.to_list()
    test_prediction_columns.remove("id")
    
    # Seleccionar las columnas para predicción
    test_prediction_data = test_df[test_prediction_columns]

    # Realizar la predicción
    predictions = model.predict(test_prediction_data)

    # Crear el DataFrame de las predicciones con 'id' y 'Accept'
    prediction_df = pd.DataFrame({
        "id": test_ids,
        "Accept": predictions
    })

    # Guardar las predicciones en un archivo CSV
    prediction_df.to_csv(f"{submissions_folder}/{submission_name}", sep=",", index=False)

    print(f"✅ Submission generada correctamente: '{submissions_folder}/{submission_name}'")

# Llamada a la función para generar la submission
create_submission(
    model=ensemble_model,
    test_df=df_test,
    submissions_folder='.',  # Reemplazar con la ruta correcta
    submission_name=f"voting_classifier_submission_{datetime.datetime.now().strftime('%Y%m%d')}.csv"
)
